In [1]:
# The code was removed by Watson Studio for sharing.

In [2]:
! pip install featuretools

     |████████████████████████████████| 276kB 9.2MB/s eta 0:00:01
     |████████████████████████████████| 71kB 20.5MB/s eta 0:00:01
  Found existing installation: tqdm 4.31.1
    Uninstalling tqdm-4.31.1:
      Successfully uninstalled tqdm-4.31.1


In [3]:
# Load Package
# import numpy as np
import pandas as pd
import seaborn as sns

In [4]:
import featuretools as ft

In [5]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import (
    OneHotEncoder,
    StandardScaler,
    LabelEncoder,
    OrdinalEncoder,
)
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import cross_val_score, RandomizedSearchCV

In [6]:
# Load data
train_df = project.get_file("train.csv")
test_df = project.get_file("test.csv")
train_df= pd.read_csv(train_df)
test_df = pd.read_csv(test_df)

In [7]:
# Save data
target = train_df[["Survived"]]
submission = test_df[["PassengerId"]]

In [8]:
# Join and Clean
combine = pd.concat([train_df, test_df])

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:2: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  from ipykernel import kernelapp as app


In [9]:
# EDA
combine.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1309 entries, 0 to 417
Data columns (total 12 columns):
Age            1046 non-null float64
Cabin          295 non-null object
Embarked       1307 non-null object
Fare           1308 non-null float64
Name           1309 non-null object
Parch          1309 non-null int64
PassengerId    1309 non-null int64
Pclass         1309 non-null int64
Sex            1309 non-null object
SibSp          1309 non-null int64
Survived       891 non-null float64
Ticket         1309 non-null object
dtypes: float64(3), int64(4), object(5)
memory usage: 132.9+ KB


In [10]:
combine.columns

Index(['Age', 'Cabin', 'Embarked', 'Fare', 'Name', 'Parch', 'PassengerId',
       'Pclass', 'Sex', 'SibSp', 'Survived', 'Ticket'],
      dtype='object')

In [11]:
mapping = {'Mlle': 'Miss', 'Major': 'Mr', 'Col': 'Mr', 'Sir': 'Mr', 'Don': 'Mr', 'Mme': 'Miss',
          'Jonkheer': 'Mr', 'Lady': 'Mrs', 'Capt': 'Mr', 'Countess': 'Mrs', 'Ms': 'Miss', 'Dona': 'Mrs'}

In [12]:
combine['Title'] = combine.Name.apply(lambda x: x.split(".")[0].split(",")[1].strip()).replace(mapping)

In [13]:
combine.drop(['Cabin', 'Ticket', 'Name'], axis=1, inplace=True)

In [14]:
# combine['Sex2'] = combine['Sex'].apply(lambda x: 0 if x=='female' else 1)

In [15]:
# class ModifiedLabelEncoder(LabelEncoder):

#     def fit_transform(self, y, *args, **kwargs):
#         return super().fit_transform(y)

#     def transform(self, y, *args, **kwargs):
#         return super().transform(y)

In [16]:
categorical_transformer = Pipeline(steps=[
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("encode", OrdinalEncoder()),
])

numeric_transformer = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
])

In [17]:
combine[['Sex','Embarked', 'Title']] = categorical_transformer.fit_transform(combine[['Sex', 'Embarked', 'Title']])

In [18]:
combine[['Age', 'Fare']] = numeric_transformer.fit_transform(combine[['Age', 'Fare']])

In [19]:
es = ft.EntitySet(id = 'titanic_data')

es = es.entity_from_dataframe(entity_id = 'combine', dataframe = combine.drop(['Survived'], axis=1), 
                              variable_types = 
                              {
                                  'Embarked': ft.variable_types.Categorical,
                                  'Sex': ft.variable_types.Boolean,
                                  'Title': ft.variable_types.Categorical
                              },
                              index = 'PassengerId')

es

Entityset: titanic_data
  Entities:
    combine [Rows: 1309, Columns: 9]
  Relationships:
    No relationships

In [20]:
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Embarked', index='Embarked')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Sex', index='Sex')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Title', index='Title')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Pclass', index='Pclass')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='Parch', index='Parch')
es = es.normalize_entity(base_entity_id='combine', new_entity_id='SibSp', index='SibSp')
es

Entityset: titanic_data
  Entities:
    combine [Rows: 1309, Columns: 9]
    Embarked [Rows: 3, Columns: 1]
    Sex [Rows: 2, Columns: 1]
    Title [Rows: 7, Columns: 1]
    Pclass [Rows: 3, Columns: 1]
    Parch [Rows: 8, Columns: 1]
    SibSp [Rows: 7, Columns: 1]
  Relationships:
    combine.Embarked -> Embarked.Embarked
    combine.Sex -> Sex.Sex
    combine.Title -> Title.Title
    combine.Pclass -> Pclass.Pclass
    combine.Parch -> Parch.Parch
    combine.SibSp -> SibSp.SibSp

In [21]:
primitives = ft.list_primitives()
pd.options.display.max_colwidth = 100
primitives[primitives['type'] == 'aggregation'].head(primitives[primitives['type'] == 'aggregation'].shape[0])

,name,type,dask_compatible,description
0,n_most_common,aggregation,False,Determines the `n` most common elements.
1,sum,aggregation,True,"Calculates the total addition, ignoring `NaN`."
2,num_true,aggregation,True,Counts the number of `True` values.
3,all,aggregation,True,Calculates if all values are 'True' in a list.
4,last,aggregation,False,Determines the last value in a list.
5,count,aggregation,True,"Determines the total number of values, excluding `NaN`."
6,std,aggregation,True,"Computes the dispersion relative to the mean value, ignoring `NaN`."
7,num_unique,aggregation,True,"Determines the number of distinct values, ignoring `NaN` values."
8,median,aggregation,False,Determines the middlemost number in a list of values.
9,min,aggregation,True,"Calculates the smallest value, ignoring `NaN` values."


In [22]:
primitives[primitives['type'] == 'transform'].head(primitives[primitives['type'] == 'transform'].shape[0])

,name,type,dask_compatible,description
22,greater_than_equal_to,transform,True,Determines if values in one list are greater than or equal to another list.
23,subtract_numeric_scalar,transform,True,Subtract a scalar from each element in the list.
24,divide_numeric_scalar,transform,True,Divide each element in the list by a scalar.
25,divide_by_feature,transform,True,Divide a scalar by each value in the list.
26,less_than,transform,True,Determines if values in one list are less than another list.
27,latitude,transform,False,Returns the first tuple value in a list of LatLong tuples.
28,greater_than,transform,True,Determines if values in one list are greater than another list.
29,cum_min,transform,False,Calculates the cumulative minimum.
30,minute,transform,True,Determines the minutes value of a datetime.
31,not,transform,True,Negates a boolean value.


In [38]:
features, feature_names = ft.dfs(entityset = es, 
                                 target_entity = 'combine', 
#                                  trans_primitives=['subtract_numeric', 'add_numeric', 'divide_numeric', 'multiply_numeric'],
                                 max_depth = 2)

In [39]:
feature_names

[<Feature: Age>,
 <Feature: Fare>,
 <Feature: Parch>,
 <Feature: Pclass>,
 <Feature: SibSp>,
 <Feature: Embarked>,
 <Feature: Sex>,
 <Feature: Title>,
 <Feature: Embarked.SUM(combine.Fare)>,
 <Feature: Embarked.SUM(combine.Age)>,
 <Feature: Embarked.STD(combine.Fare)>,
 <Feature: Embarked.STD(combine.Age)>,
 <Feature: Embarked.MAX(combine.Fare)>,
 <Feature: Embarked.MAX(combine.Age)>,
 <Feature: Embarked.SKEW(combine.Fare)>,
 <Feature: Embarked.SKEW(combine.Age)>,
 <Feature: Embarked.MIN(combine.Fare)>,
 <Feature: Embarked.MIN(combine.Age)>,
 <Feature: Embarked.MEAN(combine.Fare)>,
 <Feature: Embarked.MEAN(combine.Age)>,
 <Feature: Embarked.COUNT(combine)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Pclass)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Title)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Parch)>,
 <Feature: Embarked.NUM_UNIQUE(combine.SibSp)>,
 <Feature: Embarked.NUM_UNIQUE(combine.Sex)>,
 <Feature: Embarked.MODE(combine.Pclass)>,
 <Feature: Embarked.MODE(combine.Title)>,
 <Feature:

In [40]:
len(feature_names)

146

In [41]:
features.isnull().sum()

Age                                    0
Fare                                   0
Parch                                  0
Pclass                                 0
SibSp                                  0
Embarked                               0
Sex                                    0
Title                                  0
Embarked.SUM(combine.Fare)             0
Embarked.SUM(combine.Age)              0
Embarked.STD(combine.Fare)             0
Embarked.STD(combine.Age)              0
Embarked.MAX(combine.Fare)             0
Embarked.MAX(combine.Age)              0
Embarked.SKEW(combine.Fare)            0
Embarked.SKEW(combine.Age)             0
Embarked.MIN(combine.Fare)             0
Embarked.MIN(combine.Age)              0
Embarked.MEAN(combine.Fare)            0
Embarked.MEAN(combine.Age)             0
Embarked.COUNT(combine)                0
Embarked.NUM_UNIQUE(combine.Pclass)    0
Embarked.NUM_UNIQUE(combine.Title)     0
Embarked.NUM_UNIQUE(combine.Parch)     0
Embarked.NUM_UNI

In [55]:
class RemoveLowInfo(BaseEstimator, TransformerMixin):
    def __init__(self, threshold):
        self.threshold = threshold

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        keep = [column for column in df.columns if df[column].value_counts(normalize=True).reset_index(drop=True)[0]<self.threshold]
        return df[keep]

In [43]:
from sklearn.preprocessing import Imputer, OneHotEncoder, StandardScaler, FunctionTransformer

In [52]:
impute_median = FunctionTransformer(lambda x: x.fillna(x.median()), validate=False)

In [60]:
normalize = FunctionTransformer(lambda x: (x-x.mean())/x.std(), validate=False)

In [45]:
from sklearn.decomposition import PCA

In [61]:
transformer = Pipeline([
    ("imputer", impute_median),
    ('removelowinfo',RemoveLowInfo(threshold=.95)),
    ('scaler', normalize),
])

In [63]:
clean_features = transformer.fit_transform(features)

In [71]:
from sklearn.linear_model  import LogisticRegression, SGDClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier, BaggingClassifier, VotingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC, LinearSVC
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
import xgboost as xgb
import lightgbm as lgb
# import catboost as cgb

In [72]:
! pip install lightgbm

In [84]:
methods = [
           ('logistic', LogisticRegression(solver='lbfgs')), 
#            ('sgd', SGDClassifier()), 
           ('tree', DecisionTreeClassifier()),
           ('bag', BaggingClassifier()),
           ('xgb', xgb.XGBClassifier(max_depth=3)),
           ('lgb', lgb.LGBMClassifier(max_depth=3)),
#            ('cgb', cgb.CatBoostClassifier(max_depth=3,silent=True)),
           ('ada', AdaBoostClassifier()),
           ('gbm', GradientBoostingClassifier()),
           ('rf', RandomForestClassifier(n_estimators=100)),
#            ('svc', LinearSVC()),
#            ('rbf', SVC()),
           ('nb', Pipeline([('pca', PCA()), ('gnb', GaussianNB())])),
           ('nn', MLPClassifier()),
           ('knn', KNeighborsClassifier()),
          ]


ensemble = VotingClassifier(
        methods,
        voting='soft', 
#         weights=[1,1,1,1,2,2,1,1],
#         flatten_transform=True,
)

clf = Pipeline([
#          ('transformer', transformer),
     ('ensemble', ensemble),
])

clf.fit(clean_features.iloc[:train_df.shape[0],:], target)

/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/opt/conda/envs/Python36/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:562: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


Pipeline(memory=None,
     steps=[('ensemble', VotingClassifier(estimators=[('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='lbfgs',
          tol=0.0001, ve...  weights='uniform'))],
         flatten_transform=None, n_jobs=None, voting='soft', weights=None))])

In [87]:
submission['Survived'] = pd.DataFrame(clf.predict(clean_features.iloc[train_df.shape[0]:,:]))

In [88]:
print(submission.dtypes)

PassengerId    int64
Survived       int64
dtype: object


In [90]:
# project.save_data(file_name = "submission2.csv",data = submission.to_csv(index=False))